In [ ]:
# Automatic Lidar Data Retrieval (tiles) and Processing for Wildfire Perimeters

# This section of code automates the retrieval and processing of lidar data for wildfire perimeters.
# It iterates over a list of wildfire perimeter shapefiles, retrieves lidar data from the Planetary Computer STAC API,
# and saves the downloaded GeoTIFF files as tiles to a newly created folder, named after each fire perimeter, within an output directory.

import time
import geopandas as gpd
import pystac_client
from shapely.geometry import shape, Polygon, Point
import requests
from datetime import datetime
import os
import pandas as pd
import planetary_computer
import random

# Input to individual fire perimeters
InputFolder = "D:\\LIDARPODSPHASE1\\InputData\\IndividualPerimeters"

# Output tile locations
outputfolder = "D:\\LIDARPODSPHASE1\\IntermediateData\\"

# List of perimeter shapefiles
perimeterlist = [
    os.path.join(InputFolder, "AugustComplexFire_2020_perimeter.shp"),
    os.path.join(InputFolder, "Dixie_2021_perimeter.shp"),
    os.path.join(InputFolder, "BearNorthComplex_2020_perimeter.shp"),
    os.path.join(InputFolder, "Bush_2020_perimeter.shp"),
    os.path.join(InputFolder, "CalfCanyon_2022_perimeter.shp"),
    os.path.join(InputFolder, "CameronPeak_2020_perimeter.shp"),
    os.path.join(InputFolder, "Rafael_2021_perimeter.shp"),
    os.path.join(InputFolder, "Salt1_2020_perimeter.shp"),
    os.path.join(InputFolder, "Sugar_2021_perimeter.shp"),
    os.path.join(outputfolder, "Telegraph_2021_perimeter.shp")
]

# Year of fire for end search data for lidar
FireYear = [
    2020,
    2021,
    2020,
    2020,
    2022,
    2020,
    2021,
    2020, 
    2021,
    2021
]

# Define the collections for LIDAR - Height Above Ground and Digital Terrain Model
COLLECTIONLIST = ["3dep-lidar-hag", "3dep-lidar-dtm"]

# Accesses the Planetary Computer's STAC API using the pystac_client module
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1", modifier=planetary_computer.sign_inplace)

for fire_perimeter_shapefile, fire_year in zip(perimeterlist, FireYear):
    # Define the start and end dates for the search
    start_date = datetime(2013, 1, 1)
    end_date = datetime(fire_year - 1, 12, 31) 
    
    # Read the fire perimeter shapefile
    fire_perimeter_gdf = gpd.read_file(fire_perimeter_shapefile)

    # Create a folder for the current fire perimeter
    fire_perimeter_folder = os.path.join(outputfolder, os.path.splitext(os.path.basename(fire_perimeter_shapefile))[0])
    os.makedirs(fire_perimeter_folder, exist_ok=True)

    # Calculate the bounding box of the fire perimeter
    xmin, ymin, xmax, ymax = fire_perimeter_gdf.total_bounds
    bbox_polygon_geojson = {"type": "Polygon", "coordinates": [[
        [xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin], [xmin, ymin]
    ]]}

    # Convert the GeoJSON Polygon to a Shapely Polygon
    bbox_polygon = shape(bbox_polygon_geojson)
    buffer_distance = 0.4  # Buffer distance in degrees
    buffered_polygon = bbox_polygon.buffer(buffer_distance)
    buffered_polygon_geojson = buffered_polygon.__geo_interface__

    # Iterate over each collection
    for collection in COLLECTIONLIST:
        srch_hag = catalog.search(collections=collection, intersects=buffered_polygon_geojson,
                                  datetime=f"{start_date.isoformat()}/{end_date.isoformat()}")
        ic_hag = srch_hag.item_collection()
        
        if len(ic_hag) == 0:
            print("The ItemCollection is empty.")
        else:
            print(f"The ItemCollection contains {len(ic_hag)} items.")
        
        for i, item in enumerate(ic_hag):
            asset_url = item.assets['data'].href
            output_tif_path = os.path.join(fire_perimeter_folder, f"{os.path.splitext(os.path.basename(fire_perimeter_shapefile))[0]}_{collection}_{i}.tiff")
            response = requests.get(asset_url)
            if response.status_code == 200:
                with open(output_tif_path, 'wb') as output_file:
                    output_file.write(response.content)
                print(f"GeoTIFF file '{output_tif_path}' has been downloaded.")
            else:
                print(f"Failed to download the asset.")
            time.sleep(3)


# Define the pattern to make it more interesting with randomness
def print_random_pattern(rows, cols):
    symbols = ["D", "O", "N", "E", "!"]  # List of symbols to choose from
    for _ in range(rows):
        pattern = ''.join(random.choice(symbols) for _ in range(cols))  # Generate a random pattern for each row
        print(pattern)

# Customize the parameters here
rows = 20  # Number of lines
cols = 100  # Number of symbols in each line

# Call the function with the customized parameters
print_random_pattern(rows, cols)

In [ ]:
# Merging LiDAR Data 

# This script automates the processing of LiDAR data 
# stored in multiple directories within a specified output folder. 
# It first merges data (HAG and DTM) for each fire perimeter then saves 
# merged files to an output directory (OutputMerge)

import os
import glob
import subprocess
import random

# Output folder where merged files will be saved
OutputMerge = "D:\\LIDARPODSPHASE1\\OutputData\\"

# Find all subdirectories in the output folder
fire_perimeter_folders = [f.path for f in os.scandir(outputfolder) if f.is_dir()]

for fire_perimeter_folder in fire_perimeter_folders:
    # Extract the fire perimeter name from the folder path
    fire_perimeter_name = os.path.basename(fire_perimeter_folder)
    
    # Find all TIFF files that contain 'hag' in the filename within the fire perimeter folder
    hag_tifs = glob.glob(os.path.join(fire_perimeter_folder, "*3dep-lidar-hag*.tiff"))

    # Prepare the output file path for HAG
    output_tif_hag = os.path.join(OutputMerge, f"merged_{fire_perimeter_name}_hag.tif")

    # Prepare the gdal_merge command for HAG
    merge_command_hag = [
        "python",
        "C:\\Users\\magst\\anaconda3\\envs\\LIDAR\\Scripts\\gdal_merge.py",
        "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
        "-o", output_tif_hag,
        "-n", "-9999",
        "-a_nodata", "-9999",
    ] + hag_tifs

    # Run the gdal_merge command for HAG and capture the output
    process_hag = subprocess.run(merge_command_hag, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Check if the command for HAG was successful
    if process_hag.returncode != 0:
        # An error occurred, print the error
        print(f"Error occurred while merging TIFF files HAG for {fire_perimeter_name}:")
        print(process_hag.stderr)
    else:
        print(f"TIFF files merged successfully for HAG for {fire_perimeter_name}.")

    # Find all TIFF files that contain 'dtm' in the filename within the fire perimeter folder
    dtm_tifs = glob.glob(os.path.join(fire_perimeter_folder, "*3dep-lidar-dtm*.tiff"))

    # Prepare the output file path for DTM
    output_tif_dtm = os.path.join(OutputMerge, f"merged_{fire_perimeter_name}_dtm.tif")

    # Prepare the gdal_merge command for DTM
    merge_command_dtm = [
        "python",
        "C:\\Users\\magst\\anaconda3\\envs\\LIDAR\\Scripts\\gdal_merge.py",
        "--config", "CHECK_DISK_FREE_SPACE", "FALSE",
        "-o", output_tif_dtm,
        "-n", "-9999",
        "-a_nodata", "-9999",
    ] + dtm_tifs

    # Run the gdal_merge command for DTM and capture the output
    process_dtm = subprocess.run(merge_command_dtm, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Check if the command for DTM was successful
    if process_dtm.returncode != 0:
        # An error occurred, print the error
        print(f"Error occurred while merging TIFF files DTM for {fire_perimeter_name}:")
        print(process_dtm.stderr)
    else:
        print(f"TIFF files merged successfully for DTM for {fire_perimeter_name}.")

print("All MERGING processed")


In [ ]:
# SLOPE LOOP ALL FIRES

# This code ingests DTM merged files for each fire perimeter and 
# generates a slope layer and exports to the merged file location

from osgeo import gdal, osr
import numpy as np
import glob
import gc
import os 

# Output Merged DTM location
#OutputMerge = "D:\\LIDARPODSPHASE1\\OutputData\\"

# Find all DTM files in the directory
dtm_files = glob.glob(os.path.join(OutputMerge, "*dtm.tif"))

def calculate_slope(elevation, no_data_value):
    # Replace no-data values with NaN for slope calculation
    elevation[elevation == no_data_value] = np.nan

    # Calculate the slope
    x, y = np.gradient(elevation, edge_order=2)
    slope = np.sqrt(x**2 + y**2)
    slope = np.arctan(slope) * 180.0 / np.pi  # Convert to degrees

    # Replace NaN back with no-data value
    slope[np.isnan(slope)] = no_data_value
    return slope

for dtm_file in dtm_files:
    # Open the DTM raster file
    dtm = gdal.Open(dtm_file)
    band = dtm.GetRasterBand(1)

    # Get the size and georeferencing of the input raster
    xsize, ysize = band.XSize, band.YSize
    geotransform = dtm.GetGeoTransform()
    projection = dtm.GetProjection()

    # Define the data type and no data value for the output raster
    data_type = gdal.GDT_Float32
    no_data_value = -9999

    # Define the output slope file name
    output_slope_file = dtm_file.replace("_dtm.tif", "_slope.tif")

    # Create an output raster file
    driver = gdal.GetDriverByName('GTiff')
    out_raster = driver.Create(output_slope_file, xsize, ysize, 1, data_type)
    out_raster.SetGeoTransform(geotransform)
    out_raster.SetProjection(projection)
    out_band = out_raster.GetRasterBand(1)
    out_band.SetNoDataValue(no_data_value)
    out_band.FlushCache()

    # Define the size of the chunks
    chunk_size = 1000  # Adjust based on your system's memory

    total_chunks = ((xsize - 1) // chunk_size + 1) * ((ysize - 1) // chunk_size + 1)
    processed_chunks = 0
    next_progress_threshold = 10

    # Process the raster in chunks and write to output file
    for i in range(0, xsize, chunk_size):
        for j in range(0, ysize, chunk_size):
            xoff = i
            yoff = j
            win_xsize = min(chunk_size, xsize - i)
            win_ysize = min(chunk_size, ysize - j)
    
            elevation = band.ReadAsArray(xoff, yoff, win_xsize, win_ysize)
            slope_chunk = calculate_slope(elevation, no_data_value)
            
            processed_chunks += 1
            progress = (processed_chunks / total_chunks) * 100
            
            if progress >= next_progress_threshold:
                print(f"Processing... {progress:.2f}% complete")
                next_progress_threshold += 10
    
            # Write the slope_chunk to the output raster
            out_band.WriteArray(slope_chunk, xoff, yoff)
    
            gc.collect()
    # Close the output raster
    out_band.FlushCache()
    out_raster = None

    print(f"Slope raster saved to {output_slope_file}")

    # Collect garbage after processing each file
    gc.collect()

print("All SLOPE rasters processed")


In [ ]:
# TRI LOOP ALL FIRES

# This code processes Digital Terrain Model (DTM) merged files for each fire perimeter 
# and generates a Terrain Ruggedness Index (TRI) layer.

from osgeo import gdal, osr
import numpy as np
import glob
import gc
import os 

# Output Merged DTM location
#OutputMerge = "D:\\LIDARPODSPHASE1\\OutputData\\"

# Find all DTM files in the directory
dtm_files = glob.glob(os.path.join(OutputMerge, "*dtm.tif"))

def calculate_TRI(elevation, no_data_value):
    
    elevation[elevation == no_data_value] = np.nan

    diffs = []
    for dy in range(-1, 2):
        for dx in range(-1, 2):
            if dy == 0 and dx == 0:
                continue
            
            # Calculate differences with proper edge handling
            diff = np.empty_like(elevation)
            diff.fill(np.nan)  # Fill with NaNs

            # Define the slice ranges to avoid edge wrapping
            sy = slice(max(0, -dy), min(elevation.shape[0], elevation.shape[0] - dy))
            sx = slice(max(0, -dx), min(elevation.shape[1], elevation.shape[1] - dx))
            tsy = slice(max(0, dy), min(elevation.shape[0], elevation.shape[0] + dy))
            tsx = slice(max(0, dx), min(elevation.shape[1], elevation.shape[1] + dx))

            # Compute the difference
            diff[tsy, tsx] = elevation[tsy, tsx] - elevation[sy, sx]
            diffs.append(diff)

    # Calculate the root mean square of the differences
    diffs = np.array(diffs)
    rms_diff = np.sqrt(np.nanmean(diffs ** 2, axis=0))

    rms_diff[np.isnan(rms_diff)] = no_data_value
    return rms_diff

for dtm_file in dtm_files:
    # Open the DTM raster file
    dtm = gdal.Open(dtm_file)
    band = dtm.GetRasterBand(1)

    # Get the size and georeferencing of the input raster
    xsize, ysize = band.XSize, band.YSize
    geotransform = dtm.GetGeoTransform()
    projection = dtm.GetProjection()

    # Define the data type and no data value for the output raster
    data_type = gdal.GDT_Float32
    no_data_value = -9999

    # Define the output TRI file name
    output_tri_file = dtm_file.replace("_dtm.tif", "_tri.tif")

    # Create an output raster file
    driver = gdal.GetDriverByName('GTiff')
    out_raster = driver.Create(output_tri_file, xsize, ysize, 1, data_type)
    out_raster.SetGeoTransform(geotransform)
    out_raster.SetProjection(projection)
    out_band = out_raster.GetRasterBand(1)
    out_band.SetNoDataValue(no_data_value)
    out_band.FlushCache()

    # Define the size of the chunks
    chunk_size = 1000  # Adjust based on your system's memory

    total_chunks = ((xsize - 1) // chunk_size + 1) * ((ysize - 1) // chunk_size + 1)
    processed_chunks = 0
    next_progress_threshold = 10

    # Process the raster in chunks and write to output file
    for i in range(0, xsize, chunk_size):
        for j in range(0, ysize, chunk_size):
            xoff = i
            yoff = j
            win_xsize = min(chunk_size, xsize - i)
            win_ysize = min(chunk_size, ysize - j)
    
            elevation = band.ReadAsArray(xoff, yoff, win_xsize, win_ysize)
            TRI_chunk = calculate_TRI(elevation, no_data_value)
            
            processed_chunks += 1
            progress = (processed_chunks / total_chunks) * 100
            
            if progress >= next_progress_threshold:
                print(f"Processing... {progress:.2f}% complete")
                next_progress_threshold += 10
    
            # Write the TRI_chunk to the output raster
            out_band.WriteArray(TRI_chunk, xoff, yoff)
    
            gc.collect()
    # Close the output raster
    out_band.FlushCache()
    out_raster = None

    print(f"TRI raster saved to {output_tri_file}")

    # Collect garbage after processing each file
    gc.collect()

print("All TRI rasters processed")
